## Import libraries

In [16]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

## Data location

In [17]:
data_loc = "S://Docs//Personal//MAEVE//Data//"

## Pull data

In [18]:
path = data_loc + "BTC_price_1h.csv"
df = pd.read_csv(path)

print(f"Data shape: {df.shape}")

print(f"Date range: {df.Datetime.min()} - {df.Datetime.max()}")

Data shape: (17017, 7)
Date range: 2021-02-01 00:00:00+00:00 - 2023-01-20 13:00:00+00:00


## Custom functions

In [19]:
def calc_MA(df, timeperiod):
    df[f'MA{timeperiod}'] = df['Close'].rolling(window=timeperiod).mean()
    return df

## Calculate moving averages

In [20]:
# Calculate MA
MALst = [12, 20, 24, 30, 40, 48, 50, 60, 100, 200]

for MA in MALst:
    df = calc_MA(df, MA)

df.shape


(17017, 17)

In [21]:
# Initialize the strategy variables
current_position = None  # "buy" or "sell"
cash = 100  # Starting cash
shares = 0  # Starting shares

# Iterate over the rows of the dataframe
for index, row in df.iterrows():
    # Calculate the MA20 and MA40
    MA20 = row['MA20']
    MA40 = row['MA40']

    # Check if the MA20 is higher than the MA40
    if MA20 > MA40:
        # If we're not currently holding any shares, buy shares
        if current_position != "buy":
            shares = cash / row['Close']
            cash = 0
            current_position = "buy"
    # Check if the MA20 is lower than the MA40
    elif MA20 < MA40:
        # If we're currently holding shares, sell them
        if current_position == "buy":
            cash = shares * row['Close']
            shares = 0
            current_position = "sell"

# Print the final cash and shares value
print("Final cash value:", cash)
print("Final shares value:", shares)


,Datetime,Open,High,Low,Close,Adj Close,Volume,MA12,MA20,MA24,MA30,MA40,MA48,MA50,MA60,MA100,MA200
0,2021-02-01 00:00:00+00:00,33114.578125,33114.578125,32384.228516,32551.958984,32551.958984,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-02-01 01:00:00+00:00,32556.548828,33470.128906,32556.548828,33450.535156,33450.535156,1724944384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-02-01 02:00:00+00:00,33448.257812,33769.785156,33331.187500,33655.015625,33655.015625,458350592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-02-01 03:00:00+00:00,33654.921875,33712.281250,33531.136719,33562.886719,33562.886719,782512128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-02-01 04:00:00+00:00,33567.050781,33815.316406,33484.789062,33563.953125,33563.953125,1084145664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2021-02-01 05:00:00+00:00,33562.117188,33778.808594,33492.042969,33766.605469,33766.605469,640532480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2021-02-01 06:00:00+00:00,33772.078125,34185.832031,33391.039062,33456.828125,33456.828125,1342312448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2021-02-01 07:00:00+00:00,33536.406250,33804.683594,33458.613281,33749.070312,33749.070312,1124499456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2021-02-01 08:00:00+00:00,33740.582031,34253.121094,33656.781250,34249.644531,34249.644531,1669554176,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2021-02-01 09:00:00+00:00,34277.062500,34638.214844,34277.062500,34341.160156,34341.160156,2075963392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
